In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    FFF_Train_Decile2 = pd.read_pickle("FFF_Train_Decile2.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile2.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile2, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 2')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 4s 1ms/step - loss: 0.4988 - acc: 0.7718 - val_loss: 0.4812 - val_acc: 0.7749
Epoch 36/75
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4996 - acc: 0.7699 - val_loss: 0.4810 - val_acc: 0.7762
Epoch 37/75
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4942 - acc: 0.7732 - val_loss: 0.4809 - val_acc: 0.7749

Epoch 00037: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 38/75
3055/3055 [==============================] - 4s 1ms/step - loss: 0.5030 - acc: 0.7751 - val_loss: 0.4808 - val_acc: 0.7762
Epoch 39/75
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4980 - acc: 0.7702 - val_loss: 0.4810 - val_acc: 0.7762
Epoch 40/75
3055/3055 [==============================] - 4s 1ms/step - loss: 0.5054 - acc: 0.7715 - val_loss: 0.4809 - val_acc: 0.7762
Epoch 41/75
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4980 - acc: 0.7702 - val_loss: 0.4810 - val_acc: 0.7762
Ep

3055/3055 [==============================] - 4s 1ms/step - loss: 0.4829 - acc: 0.7712 - val_loss: 0.4799 - val_acc: 0.7736
Epoch 20/75
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4885 - acc: 0.7735 - val_loss: 0.4792 - val_acc: 0.7709
Epoch 21/75
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4866 - acc: 0.7745 - val_loss: 0.4797 - val_acc: 0.7696

Epoch 00021: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 22/75
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4915 - acc: 0.7686 - val_loss: 0.4795 - val_acc: 0.7709
Epoch 23/75
3055/3055 [==============================] - 5s 1ms/step - loss: 0.4866 - acc: 0.7682 - val_loss: 0.4790 - val_acc: 0.7709
Epoch 24/75
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4816 - acc: 0.7764 - val_loss: 0.4793 - val_acc: 0.7709
Epoch 25/75
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4842 - acc: 0.7791 - val_loss: 0.4794 - v

3055/3055 [==============================] - 4s 1ms/step - loss: 0.4900 - acc: 0.7666 - val_loss: 0.4763 - val_acc: 0.7683
Epoch 26/125
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4925 - acc: 0.7633 - val_loss: 0.4763 - val_acc: 0.7683
Epoch 27/125
3055/3055 [==============================] - 5s 1ms/step - loss: 0.4876 - acc: 0.7774 - val_loss: 0.4766 - val_acc: 0.7683
Epoch 28/125
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4853 - acc: 0.7682 - val_loss: 0.4766 - val_acc: 0.7683
Epoch 29/125
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4904 - acc: 0.7676 - val_loss: 0.4767 - val_acc: 0.7670
Epoch 30/125
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4889 - acc: 0.7748 - val_loss: 0.4762 - val_acc: 0.7696
Epoch 31/125
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4903 - acc: 0.7699 - val_loss: 0.4766 - val_acc: 0.7683
Epoch 32/125
3055/3055 [==============================] - 4s 

3055/3055 [==============================] - 4s 1ms/step - loss: 0.5043 - acc: 0.7738 - val_loss: 0.4815 - val_acc: 0.7866
Epoch 67/100
3055/3055 [==============================] - 5s 1ms/step - loss: 0.5029 - acc: 0.7614 - val_loss: 0.4815 - val_acc: 0.7880
Epoch 68/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.5030 - acc: 0.7656 - val_loss: 0.4815 - val_acc: 0.7866
Epoch 69/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4981 - acc: 0.7699 - val_loss: 0.4816 - val_acc: 0.7880
Epoch 70/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.5042 - acc: 0.7614 - val_loss: 0.4815 - val_acc: 0.7866
Epoch 71/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4994 - acc: 0.7696 - val_loss: 0.4816 - val_acc: 0.7880
Epoch 72/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.5013 - acc: 0.7699 - val_loss: 0.4814 - val_acc: 0.7866
Epoch 73/100
3055/3055 [==============================] - 4s 

Epoch 5/125
3055/3055 [==============================] - 5s 2ms/step - loss: 0.5872 - acc: 0.7011 - val_loss: 0.5557 - val_acc: 0.7330
Epoch 6/125
3055/3055 [==============================] - 4s 1ms/step - loss: 0.5670 - acc: 0.7188 - val_loss: 0.5357 - val_acc: 0.7448
Epoch 7/125
3055/3055 [==============================] - 4s 1ms/step - loss: 0.5555 - acc: 0.7257 - val_loss: 0.5225 - val_acc: 0.7526
Epoch 8/125
3055/3055 [==============================] - 4s 1ms/step - loss: 0.5449 - acc: 0.7430 - val_loss: 0.5140 - val_acc: 0.7657
Epoch 9/125
3055/3055 [==============================] - 5s 2ms/step - loss: 0.5384 - acc: 0.7463 - val_loss: 0.5084 - val_acc: 0.7618
Epoch 10/125
3055/3055 [==============================] - 4s 1ms/step - loss: 0.5348 - acc: 0.7656 - val_loss: 0.5053 - val_acc: 0.7657
Epoch 11/125
3055/3055 [==============================] - 4s 1ms/step - loss: 0.5300 - acc: 0.7578 - val_loss: 0.5010 - val_acc: 0.7696
Epoch 12/125
3055/3055 [=============================

Epoch 65/125
3055/3055 [==============================] - 5s 2ms/step - loss: 0.4988 - acc: 0.7745 - val_loss: 0.4831 - val_acc: 0.7762
Epoch 66/125
3055/3055 [==============================] - 5s 2ms/step - loss: 0.4916 - acc: 0.7755 - val_loss: 0.4830 - val_acc: 0.7762
Epoch 67/125
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4923 - acc: 0.7735 - val_loss: 0.4830 - val_acc: 0.7762
Epoch 68/125
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4974 - acc: 0.7712 - val_loss: 0.4829 - val_acc: 0.7762
Epoch 69/125
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4954 - acc: 0.7768 - val_loss: 0.4829 - val_acc: 0.7762
Epoch 70/125
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4986 - acc: 0.7686 - val_loss: 0.4826 - val_acc: 0.7749
Epoch 71/125
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4944 - acc: 0.7722 - val_loss: 0.4829 - val_acc: 0.7762
Epoch 72/125
3055/3055 [========================

3055/3055 [==============================] - 2s 612us/step - loss: 0.4883 - acc: 0.7702 - val_loss: 0.4779 - val_acc: 0.7696
Epoch 53/100
3055/3055 [==============================] - 2s 623us/step - loss: 0.4926 - acc: 0.7748 - val_loss: 0.4783 - val_acc: 0.7723
Epoch 54/100
3055/3055 [==============================] - 2s 602us/step - loss: 0.4832 - acc: 0.7761 - val_loss: 0.4784 - val_acc: 0.7736
Epoch 55/100
3055/3055 [==============================] - 2s 593us/step - loss: 0.4893 - acc: 0.7696 - val_loss: 0.4784 - val_acc: 0.7736
Epoch 56/100
3055/3055 [==============================] - 2s 596us/step - loss: 0.4819 - acc: 0.7741 - val_loss: 0.4784 - val_acc: 0.7736
Epoch 57/100
3055/3055 [==============================] - 2s 594us/step - loss: 0.4875 - acc: 0.7732 - val_loss: 0.4783 - val_acc: 0.7723
Epoch 58/100
3055/3055 [==============================] - 2s 595us/step - loss: 0.4904 - acc: 0.7735 - val_loss: 0.4782 - val_acc: 0.7723
Epoch 59/100
3055/3055 [=======================

3055/3055 [==============================] - 2s 576us/step - loss: 0.5019 - acc: 0.7630 - val_loss: 0.4791 - val_acc: 0.7736
Epoch 12/50
3055/3055 [==============================] - 2s 579us/step - loss: 0.4974 - acc: 0.7699 - val_loss: 0.4760 - val_acc: 0.7683
Epoch 13/50
3055/3055 [==============================] - 2s 582us/step - loss: 0.4905 - acc: 0.7725 - val_loss: 0.4837 - val_acc: 0.7657
Epoch 14/50
3055/3055 [==============================] - 2s 576us/step - loss: 0.4980 - acc: 0.7712 - val_loss: 0.4751 - val_acc: 0.7696
Epoch 15/50
3055/3055 [==============================] - 2s 577us/step - loss: 0.4888 - acc: 0.7738 - val_loss: 0.4753 - val_acc: 0.7696
Epoch 16/50
3055/3055 [==============================] - 2s 578us/step - loss: 0.4970 - acc: 0.7653 - val_loss: 0.4805 - val_acc: 0.7762
Epoch 17/50
3055/3055 [==============================] - 2s 577us/step - loss: 0.4923 - acc: 0.7669 - val_loss: 0.4795 - val_acc: 0.7696

Epoch 00017: ReduceLROnPlateau reducing learning rat

In [3]:
0.93484

0.93484